In [92]:
import time

# Start time
t0 = time.time()

In [93]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 999)
pd.set_option('max_columns', 999)

import warnings
warnings.filterwarnings('ignore')

In [94]:
df = pd.read_json('Data Analytics/contacts.json')

In [95]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [96]:
df['Id'] = df['Id'].astype(str)

# Grouping by same Email/Phone/OrderId

## Create Function

In [97]:
def group_by_col(col):
    grouped = df.copy()
    
    # Change row with whitespace to NaN then drop them
    grouped[col] = grouped[col].apply(lambda x: np.NaN if x == '' else x)
    grouped = grouped.dropna()
    
    # Get data with duplicated col only
    grouped = grouped[grouped.duplicated([col], keep=False)]
    
    # Groupby same col and combine the Id
    grouped = grouped.groupby(col)['Id'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
    return grouped

In [98]:
phone = group_by_col('Phone')
phone.head()

,Phone,Id
0,0000211995,229812 265783
1,0000398025,95070 428624
2,000069388,142031 395090
3,000070496474,107760 296641
4,0000756335,139526 288114


In [99]:
order = group_by_col('OrderId')
order.head()

,OrderId,Id
0,AAHeDQwhQuenpHpFDtYbzhRtG,13086 35886
1,AAJYFqwdgWsaQjitbggfaqoNN,217525 222796 488624
2,AALCakLTXBzGFWkkPReVvzNCh,212727 361700 454292
3,AANGeFiGESRFtzgRLqnySbyvu,51827 255333
4,AAQkojNLBHeApFCQIKQOeccFs,338630 401880 436322


In [100]:
email = group_by_col('Email')
email.head()

,Email,Id
0,AAAQbNfGbLAeOvw@hotmail.com,112387 470075
1,AADMsTZAxsOHvqnKjg@hotmail.com,19504 179125
2,AAJcFFDgfLHBkDrXoaDx@yahoo.com,20825 337532
3,AAOPaYMogBRr@gmail.com,221848 330517
4,AAPIPtjhTqBsjvnutTU@hotmail.com,254424 367012 390060


## Merge the Grouped Columns

In [101]:
merged_df = df.merge(email, how='left', on='Email')
merged_df.head()

,Id_x,Email,Phone,Contacts,OrderId,Id_y
0,0,gkzAbIy@qq.com,,1,,NaN
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,NaN
2,2,,9125983679,0,,NaN
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,NaN
4,4,,300364407,2,,NaN


In [102]:
merged_df = merged_df.rename(columns={"Id_x": "Id", "Id_y": "combined_email"})
merged_df.head(7)

,Id,Email,Phone,Contacts,OrderId,combined_email
0,0,gkzAbIy@qq.com,,1,,NaN
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,NaN
2,2,,9125983679,0,,NaN
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,NaN
4,4,,300364407,2,,NaN
5,5,,840113148,0,,NaN
6,6,hESiBgYTxMnj@hotmail.com,,4,,6 38 32871 142067


In [103]:
merged_df = merged_df.merge(phone, how='left', on='Phone')
merged_df = merged_df.rename(columns={"Id_x": "Id", "Id_y": "combined_phone"})

merged_df = merged_df.merge(order, how='left', on='OrderId')
merged_df = merged_df.rename(columns={"Id_x": "Id", "Id_y": "combined_orderid"})

# Replace NaN with empty string
merged_df['combined_email'] = merged_df['combined_email'].fillna("")
merged_df['combined_phone'] = merged_df['combined_phone'].fillna("")
merged_df['combined_orderid'] = merged_df['combined_orderid'].fillna("")

merged_df.head(10)

,Id,Email,Phone,Contacts,OrderId,combined_email,combined_phone,combined_orderid
0,0,gkzAbIy@qq.com,,1,,,,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,,,1 2458 476346
2,2,,9125983679,0,,,2 348955,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,,,
4,4,,300364407,2,,,,
5,5,,840113148,0,,,5 50,
6,6,hESiBgYTxMnj@hotmail.com,,4,,6 38 32871 142067,,
7,7,hqyunTiaXfIjhO@yahoo.com,,1,vxjYZtMUvNhtEBDruzQfjpsmX,,,
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,,,8 183160,
9,9,osIIQgAiZX@hotmail.com,,0,IqewrMXVTJqSsVklWFfOpdRNI,,,9 13 343161


In [104]:
merged_df['duplicated_indexes'] = merged_df['combined_email'] +  ' ' +  + merged_df['combined_phone'] +  ' ' +  + merged_df['combined_orderid'].fillna("")
merged_df = merged_df[['Id', 'Contacts', 'duplicated_indexes']]
merged_df.head(7)

,Id,Contacts,duplicated_indexes
0,0,1,
1,1,4,1 2458 476346
2,2,0,2 348955
3,3,0,
4,4,2,
5,5,0,5 50
6,6,4,6 38 32871 142067


In [105]:
merged_df['duplicated_indexes'].unique()

array(['  ', '  1 2458 476346', ' 2 348955 ', ...,
       '142678 143516 499987 44161 499987 ', '  204097 499990',
       '  184208 499994'], dtype=object)

# Calculate Sum of Contacts

## Single Channel

In [106]:
merged_df['duplicated_indexes'].unique()

array(['  ', '  1 2458 476346', ' 2 348955 ', ...,
       '142678 143516 499987 44161 499987 ', '  204097 499990',
       '  184208 499994'], dtype=object)

In [107]:
single_channel = merged_df[merged_df['duplicated_indexes'] == '  ']
print(single_channel.shape)
single_channel.head()

(195808, 3)


,Id,Contacts,duplicated_indexes
0,0,1,
3,3,0,
4,4,2,
7,7,1,
16,16,0,


In [108]:
single_channel.dtypes

Id                    object
Contacts               int64
duplicated_indexes    object
dtype: object

In [109]:
single_channel['Contacts'] = single_channel['Contacts'].astype(str)
single_channel['trace'] = single_channel['Id'] + ', ' + single_channel['Contacts']
single_channel = single_channel[['Id', 'trace']]
single_channel.head()

,Id,trace
0,0,"0, 1"
3,3,"3, 0"
4,4,"4, 2"
7,7,"7, 1"
16,16,"16, 0"


## Multi Channels

In [110]:
multi_channel = merged_df[merged_df['duplicated_indexes'] != '  ']
print(multi_channel.shape)
multi_channel.head(10)

(304192, 3)


,Id,Contacts,duplicated_indexes
1,1,4,1 2458 476346
2,2,0,2 348955
5,5,0,5 50
6,6,4,6 38 32871 142067
8,8,2,8 183160
9,9,0,9 13 343161
10,10,3,10 93270
11,11,0,11 244207
12,12,2,12 480595
13,13,1,13 417916 13 16708 9 13 343161


### Step by Step to Clean duplicated_indexes

### Function to Clean duplicated_indexes

In [111]:
def clean_duplicated_indexes(x):
    x = x.split()
    x = list(map(int, x))
    x = list(set(x))
    x.sort()
    x = list(map(str, x))
    x = '-'.join((x))
    return x

In [112]:
multi_channel['duplicated_indexes'] = multi_channel['duplicated_indexes'].apply(lambda x: clean_duplicated_indexes(x))
multi_channel.head()

,Id,Contacts,duplicated_indexes
1,1,4,1-2458-476346
2,2,0,2-348955
5,5,0,5-50
6,6,4,6-38-32871-142067
8,8,2,8-183160


### Step by Step to Create Sum of Contacts

In [113]:
multi_channel['duplicated_indexes'].unique()[:5]

array(['1-2458-476346', '2-348955', '5-50', '6-38-32871-142067',
       '8-183160'], dtype=object)

In [114]:
unique_idxs = multi_channel['duplicated_indexes'].unique()
len(unique_idxs)

213862

In [115]:
unique = unique_idxs[2]
x = unique.split('-')
x = list(map(int, x))
x

[5, 50]

In [116]:
# Check in original df
df[df.index.isin([5, 50])]

,Id,Email,Phone,Contacts,OrderId
5,5,,840113148,0,
50,50,TIMejlVYhfqjy@gmail.com,840113148,4,XBkNrwcWWslDvSQdhYPfCpQJo


In [117]:
df.loc[50, 'Contacts']

4

In [118]:
sum_contacts = 0

for idx in x:
    contacts = df.loc[idx, 'Contacts']
    sum_contacts += contacts

sum_contacts

4

In [119]:
# Create dictionary with unique_idxs as a key
sum_contacts_dict = {}
sum_contacts_dict[unique] = sum_contacts
sum_contacts_dict

{'5-50': 4}

### Function to Create Sum of Contacts

In [120]:
unique_idxs = multi_channel['duplicated_indexes'].unique()
len(unique_idxs)

213862

In [121]:
sum_contacts_dict = {}

for unique in unique_idxs:
    idx_list = unique.split('-')
    idx_list = list(map(int, idx_list))

    sum_contacts = 0

    for idx in idx_list:
        contacts = df.loc[idx, 'Contacts']
        sum_contacts += contacts

    sum_contacts_dict[unique] = sum_contacts

In [122]:
len(sum_contacts_dict)

213862

In [123]:
multi_channel['sum_contacts'] = multi_channel['duplicated_indexes'].map(sum_contacts_dict)
multi_channel.head()

,Id,Contacts,duplicated_indexes,sum_contacts
1,1,4,1-2458-476346,5
2,2,0,2-348955,0
5,5,0,5-50,4
6,6,4,6-38-32871-142067,10
8,8,2,8-183160,5


In [124]:
multi_channel['sum_contacts'] = multi_channel['sum_contacts'].astype(str)
multi_channel['trace'] = multi_channel['duplicated_indexes'] + ', ' + multi_channel['sum_contacts']
multi_channel = multi_channel[['Id', 'trace']]
multi_channel.head()

,Id,trace
1,1,"1-2458-476346, 5"
2,2,"2-348955, 0"
5,5,"5-50, 4"
6,6,"6-38-32871-142067, 10"
8,8,"8-183160, 5"


# Create Submission

In [125]:
final_df = pd.concat([single_channel, multi_channel])
final_df['Id'] = final_df['Id'].astype(int)
final_df.sort_values('Id', inplace=True)
final_df.head()

,Id,trace
0,0,"0, 1"
1,1,"1-2458-476346, 5"
2,2,"2-348955, 0"
3,3,"3, 0"
4,4,"4, 2"


In [126]:
final_df.rename(columns={"Id": "ticket_id", "trace": "ticket_trace/contact"}, inplace=True)
print(final_df.shape)
final_df.head(20)

(500000, 2)


,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-476346, 5"
2,2,"2-348955, 0"
3,3,"3, 0"
4,4,"4, 2"
5,5,"5-50, 4"
6,6,"6-38-32871-142067, 10"
7,7,"7, 1"
8,8,"8-183160, 5"
9,9,"9-13-343161, 1"


In [127]:
final_df.to_csv('Data Analytics/multi-channel-contacts-submission.csv', index=False)

In [128]:
t1 = time.time()
total = t1-t0
print("Notebook total running time: ", round(total, 2), " seconds")

Notebook total running time:  14.21  seconds


## Test Case

In [129]:
final_df['ticket_trace/contact'][13]

'9-13-16708-343161-417916, 5'

In [130]:
df[df.index.isin([9, 13, 16708, 343161, 417916])]

,Id,Email,Phone,Contacts,OrderId
9,9,osIIQgAiZX@hotmail.com,,0,IqewrMXVTJqSsVklWFfOpdRNI
13,13,YsKhkKGJoQSNniHcr@yahoo.com,562300766,1,IqewrMXVTJqSsVklWFfOpdRNI
16708,16708,baztAgq@gmail.com,562300766,1,uZmSoONExsyEvZXwPAaRwTBzJ
343161,343161,,6019130221,0,IqewrMXVTJqSsVklWFfOpdRNI
417916,417916,YsKhkKGJoQSNniHcr@yahoo.com,,3,lTFWCsPHyXciSbwCenwkNXOlv


In [131]:
final_df['ticket_trace/contact'][18]

'18-274269-327057, 5'

In [132]:
df[df.index.isin([18, 274269, 327057])]

,Id,Email,Phone,Contacts,OrderId
18,18,qOkuzhOuPsbltatw@qq.com,,4,zSekjOkRLyiGjvOPikTKwksFK
274269,274269,tndZkvBDwg@gmail.com,66873440472,0,zSekjOkRLyiGjvOPikTKwksFK
327057,327057,sjoEThSNaVBlYk@gmail.com,,1,zSekjOkRLyiGjvOPikTKwksFK
